In [1]:
/* 22-1 */

SELECT r.session_id, r.status, r.start_time, r.command, s.text
FROM sys.dm_exec_requests r
CROSS APPLY sys.dm_exec_sql_text(r.sql_handle) s
WHERE r.status = 'running';

(2 rows affected)

Total execution time: 00:00:00.356

session_id,status,start_time,command,text
52,running,2020-09-11 19:51:39.693,EXECUTE,"declare @MasterPath nvarchar(512) declare @LogPath nvarchar(512) declare @ErrorLog nvarchar(512) declare @ErrorLogPath nvarchar(512) declare @Slash varchar = convert(varchar, serverproperty('PathSeparator')) if (SERVERPROPERTY('EngineEdition') = 8 /* SQL Managed Instance */) begin select @MasterPath=substring(physical_name, 1, len(physical_name) - charindex(@Slash, reverse(physical_name))) from master.sys.database_files where file_id = 1 select @LogPath=substring(physical_name, 1, len(physical_name) - charindex(@Slash, reverse(physical_name))) from master.sys.database_files where file_id = 2 end else begin select @MasterPath=substring(physical_name, 1, len(physical_name) - charindex(@Slash, reverse(physical_name))) from master.sys.database_files where name=N'master' select @LogPath=substring(physical_name, 1, len(physical_name) - charindex(@Slash, reverse(physical_name))) from master.sys.database_files where name=N'mastlog' end select @ErrorLog=cast(SERVERPROPERTY(N'errorlogfilename') as nvarchar(512)) select @ErrorLogPath=IIF(@ErrorLog IS NULL, N'', substring(@ErrorLog, 1, len(@ErrorLog) - charindex(@Slash, reverse(@ErrorLog)))) declare @SmoRoot nvarchar(512) exec master.dbo.xp_instance_regread N'HKEY_LOCAL_MACHINE', N'SOFTWARE\Microsoft\MSSQLServer\Setup', N'SQLPath', @SmoRoot OUTPUT SELECT CAST(case when 'a' <> 'A' then 1 else 0 end AS bit) AS [IsCaseSensitive], @@MAX_PRECISION AS [MaxPrecision], @ErrorLogPath AS [ErrorLogPath], @SmoRoot AS [RootDirectory], SERVERPROPERTY('PathSeparator') AS [PathSeparator], CAST(FULLTEXTSERVICEPROPERTY('IsFullTextInstalled') AS bit) AS [IsFullTextInstalled], @LogPath AS [MasterDBLogPath], @MasterPath AS [MasterDBPath], SERVERPROPERTY(N'ProductVersion') AS [VersionString], CAST(SERVERPROPERTY(N'Edition') AS sysname) AS [Edition], CAST(SERVERPROPERTY(N'ProductLevel') AS sysname) AS [ProductLevel], CAST(SERVERPROPERTY('IsSingleUser') AS bit) AS [IsSingleUser], CAST(SERVERPROPERTY('EngineEdition') AS int) AS [EngineEdition], convert(sysname, serverproperty(N'collation')) AS [Collation], CAST(ISNULL(SERVERPROPERTY(N'MachineName'),N'') AS sysname) AS [NetName], CAST(ISNULL(SERVERPROPERTY('IsClustered'),N'') AS bit) AS [IsClustered], SERVERPROPERTY(N'ResourceVersion') AS [ResourceVersionString], SERVERPROPERTY(N'ResourceLastUpdateDateTime') AS [ResourceLastUpdateDateTime], SERVERPROPERTY(N'CollationID') AS [CollationID], SERVERPROPERTY(N'ComparisonStyle') AS [ComparisonStyle], SERVERPROPERTY(N'SqlCharSet') AS [SqlCharSet], SERVERPROPERTY(N'SqlCharSetName') AS [SqlCharSetName], SERVERPROPERTY(N'SqlSortOrder') AS [SqlSortOrder], SERVERPROPERTY(N'SqlSortOrderName') AS [SqlSortOrderName], SERVERPROPERTY(N'BuildClrVersion') AS [BuildClrVersionString], ISNULL(SERVERPROPERTY(N'ComputerNamePhysicalNetBIOS'),N'') AS [ComputerNamePhysicalNetBIOS], CAST(SERVERPROPERTY('IsPolyBaseInstalled') AS bit) AS [IsPolyBaseInstalled]"
53,running,2020-09-11 19:51:39.763,SELECT,"/* 22-1 */ SELECT r.session_id, r.status, r.start_time, r.command, s.text FROM sys.dm_exec_requests r CROSS APPLY sys.dm_exec_sql_text(r.sql_handle) s WHERE r.status = 'running';"


In [2]:
/* Solution 2 */
USE master;
GO
-- Create the Event Session
IF EXISTS(SELECT * 
          FROM sys.server_event_sessions 
          WHERE name='TraceIncomingQueries')
    DROP EVENT SESSION TraceIncomingQueries 
    ON SERVER;
GO
CREATE EVENT SESSION TraceIncomingQueries
ON SERVER
ADD EVENT sqlserver.sql_statement_starting(
    ACTION(sqlserver.database_name,sqlserver.nt_username,sqlserver.session_id,sqlserver.client_hostname,sqlserver.client_app_name)
WHERE sqlserver.database_name='AdventureWorks2014'
	AND sqlserver.client_app_name <> 'Microsoft SQL Server Management Studio - Transact-SQL IntelliSense'
	)
ADD TARGET package0.event_file(SET filename=N'C:\Database\XE\TraceIncomingQueries.xel')

/* start the session */
ALTER EVENT SESSION TraceIncomingQueries 
ON SERVER 
STATE = START;
GO

Commands completed successfully.

Commands completed successfully.

: Msg 25602, Level 17, State 0, Line 21
The target, "5B2DA06D-898A-43C8-9309-39BBBE93EBBD.package0.event_file", encountered a configuration error during initialization. Object cannot be added to the event session. The operating system returned error 3: 'The system cannot find the path specified.
' while creating the file 'C:\Database\XE\TraceIncomingQueries_0_132443419043740000.xel'.

Total execution time: 00:00:00.910

In [3]:
/* now run a query */
USE AdventureWorks2014;
GO

SELECT r.session_id, r.status, r.start_time, r.command, s.text
FROM sys.dm_exec_requests r
CROSS APPLY sys.dm_exec_sql_text(r.sql_handle) s
WHERE r.status = 'running';

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.168

session_id,status,start_time,command,text
53,running,2020-09-11 19:51:51.917,SELECT,"SELECT r.session_id, r.status, r.start_time, r.command, s.text FROM sys.dm_exec_requests r CROSS APPLY sys.dm_exec_sql_text(r.sql_handle) s WHERE r.status = 'running';"


In [4]:
/* Parse the event data */

use master;
GO

SELECT
event_data.value('(event/@name)[1]', 'varchar(50)') AS event_name,
    event_data.value('(event/@timestamp)[1]', 'varchar(50)') AS [TIMESTAMP],
	event_data.value('(event/action[@name="database_name"]/value)[1]', 'varchar(max)') AS DBName
	,event_data.value('(event/data[@name="statement"]/value)[1]', 'varchar(max)') AS SQLText
	,event_data.value('(event/action[@name="session_id"]/value)[1]', 'varchar(max)') AS SessionID
	,event_data.value('(event/action[@name="nt_username"]/value)[1]', 'varchar(max)') AS ExecUser
	,event_data.value('(event/action[@name="client_hostname"]/value)[1]', 'varchar(max)') AS Client_HostName,
	event_data.value('(event/action[@name="client_app_name"]/value)[1]', 'varchar(max)') AS Client_AppName
FROM(
SELECT CONVERT(XML, t2.event_data) AS event_data
 FROM (
  SELECT target_data = convert(XML, target_data)
   FROM sys.dm_xe_session_targets t
    INNER JOIN sys.dm_xe_sessions s 
        ON t.event_session_address = s.address
   WHERE t.target_name = 'event_file'
    AND s.name = 'TraceIncomingQueries') cte1
   CROSS APPLY cte1.target_data.nodes('//EventFileTarget/File') FileEvent(FileTarget)
   CROSS APPLY  sys.fn_xe_file_target_read_file(FileEvent.FileTarget.value('@name', 'varchar(1000)'), NULL, NULL, NULL) t2)
    AS evts(event_data);

Commands completed successfully.

(0 rows affected)

event_name,TIMESTAMP,DBName,SQLText,SessionID,ExecUser,Client_HostName,Client_AppName


Total execution time: 00:01:53.253